In [ ]:
import numpy as np
import pandas as pd
from time import perf_counter
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import warnings
warnings.filterwarnings("ignore")

from joblib import Memory
memory = Memory('./chache', verbose=0)

In [ ]:
# @memory.cache
def random_search(distance_matrix, current_node_index=None):
    to_visit = set(range(len(distance_matrix)))
    
    if current_node_index is None:
        current_node_index = np.random.choice(list(to_visit))
    
    current_node = current_node_index
    solution = [current_node]
    total_cost = distance_matrix[current_node, current_node]
    to_visit.remove(current_node_index)
    
    while len(solution) != len(distance_matrix) // 2:
        next_node_index = np.random.choice(list(to_visit))
        next_node = next_node_index
        total_cost += distance_matrix[current_node, next_node]
        solution.append(next_node)
        to_visit.remove(next_node_index)
        current_node = next_node
    
    # solution.append(solution[0])
    total_cost += distance_matrix[solution[-1], solution[0]]
    
    return solution, total_cost


In [ ]:
# @memory.cache
def greedy_2_regret_weighted(distance_matrix, current_node_index=None, regret_weight=0.5):
    num_nodes = len(distance_matrix)
    to_visit = set(range(num_nodes))
    
    if current_node_index is None:
        current_node_index = np.random.choice(list(to_visit))
    
    current_node = current_node_index
    solution = [current_node]
    to_visit.remove(current_node_index)
    
    while len(solution) != len(distance_matrix) // 2:
        max_weighted_sum = float('-inf')
        best_node = None
        best_insertion_point = None

        for node in to_visit:
            insertion_costs = []
            for i in range(len(solution) - 1):
                cost = distance_matrix[solution[i]][node] + distance_matrix[node][solution[i+1]] - distance_matrix[solution[i]][solution[i+1]]
                insertion_costs.append((cost, i))

            insertion_costs.append((distance_matrix[solution[-1]][node] + distance_matrix[node][solution[0]] - distance_matrix[solution[-1]][solution[0]], len(solution) - 1))

            insertion_costs.sort(key=lambda x: x[0])

            weighted_sum = 0
            if len(insertion_costs) > 1:
                regret = insertion_costs[1][0] - insertion_costs[0][0]
                objectivee = -insertion_costs[0][0]
                weighted_sum = regret_weight * regret   +    (1 - regret_weight) * objectivee

            if weighted_sum > max_weighted_sum:
                max_weighted_sum = weighted_sum
                best_node = node
                best_insertion_point = insertion_costs[0][1]

        solution.insert(best_insertion_point + 1, best_node)
        to_visit.remove(best_node)

    total_cost = sum(distance_matrix[solution[i]][solution[i+1]] for i in range(len(solution) - 1))
    total_cost += distance_matrix[solution[-1]][solution[0]]

    return solution, total_cost

In [ ]:
def calculate_euclidean_distance(node1, node2):
    return np.round(np.sqrt((node1['x'] - node2['x'])**2 + (node1['y'] - node2['y'])**2))


def get_distance_matrix(df, costs=False):
    num_nodes = df.shape[0]
    distance_matrix = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        node1 = df.iloc[i]
        for j in range(i, num_nodes):
            node2 = df.iloc[j]
            distance = calculate_euclidean_distance(node1, node2)
            if costs:
                distance_matrix[i,j] = distance + node2['cost']
                distance_matrix[j,i] = distance + node1['cost']
            else:
                distance_matrix[i,j] = distance
                distance_matrix[j,i] = distance
    return distance_matrix

In [ ]:
# @memory.cache
def get_node_moves(solution, flag):
    n = len(solution)
    moves = list()
    if flag == 'intra':
        for i in range(len(solution) - 1):
            for j in range(i+1, len(solution)):
                moves.append((solution[i], solution[j]))
    elif flag == 'inter':
        outer_nodes = list(set(range(int(n*2))) - set(solution))
        for i in range(len(solution)):
            for j in range(len(outer_nodes)):
                moves.append((solution[i], outer_nodes[j]))   
    return moves

In [ ]:
# @memory.cache
def get_edge_moves(solution):
    moves = list()
    for i in range(len(solution) - 1):
        for j in range(i + 2, len(solution) - 1):
            edge1 = (solution[i], solution[i + 1])
            edge2 = (solution[j], solution[j + 1])
            moves.append((edge1, edge2))
    return moves

In [ ]:
# @memory.cache
def compute_node_swap_delta(solution, swap, distance_matrix, costs, flag):
    new_solution = solution[:]
    n = len(new_solution)
    if flag == 'intra':
        node1, node2 = swap
        node1_idx, node2_idx = new_solution.index(node1), new_solution.index(node2)
        if node1_idx > node2_idx:
            node1, node2 = node2, node1
            node1_idx, node2_idx = node2_idx, node1_idx
        prev_node1, next_node1 = new_solution[node1_idx-1], new_solution[(node1_idx+1)%n]
        prev_node2, next_node2 = new_solution[node2_idx-1], new_solution[(node2_idx+1)%n]
        if (node1_idx+1)%n == node2_idx or (node2_idx+1)%n == node1_idx:
            if node1_idx==0 and node2_idx==len(new_solution)-1:
                prev_node1, node1, node2, next_node2 = prev_node2, node2, node1, next_node1
            elif node1_idx > node2_idx:
                node1_idx, node2_idx = node2_idx, node1_idx
                prev_node1, next_node2 = prev_node2, next_node1
                node1, node2 = node2, node1
            old_cost = distance_matrix[prev_node1][node1] + distance_matrix[node1][node2] + distance_matrix[node2][next_node2]
            new_cost = distance_matrix[prev_node1][node2] + distance_matrix[node2][node1] + distance_matrix[node1][next_node2]
        else:
            old_cost = distance_matrix[prev_node1][node1] + distance_matrix[node1][next_node1] + \
                       distance_matrix[prev_node2][node2] + distance_matrix[node2][next_node2]
            new_cost = distance_matrix[prev_node1][node2] + distance_matrix[node2][next_node1] + \
                       distance_matrix[prev_node2][node1] + distance_matrix[node1][next_node2]
        delta = old_cost - new_cost

        new_solution[node1_idx], new_solution[node2_idx] = new_solution[node2_idx], new_solution[node1_idx]
        
    elif flag == 'inter':
        solution_node, outer_node = swap
        solution_node_idx = new_solution.index(solution_node)
        prev_solution_node, next_solution_node = new_solution[solution_node_idx-1], new_solution[(solution_node_idx+1)%n]

        old_cost = distance_matrix[prev_solution_node][solution_node] + distance_matrix[solution_node][next_solution_node] + \
                   costs[solution_node]
        new_cost = distance_matrix[prev_solution_node][outer_node] + distance_matrix[outer_node][next_solution_node] + \
                   costs[outer_node]
        
        delta = old_cost - new_cost
        new_solution[solution_node_idx] = outer_node

    return delta, new_solution

In [ ]:
# @memory.cache
def compute_edge_swap_delta(solution, swap, distance_matrix):
    (edge1_start, edge1_end), (edge2_start, edge2_end) = swap
    idx_edge1_start = solution.index(edge1_start)
    idx_edge2_start = solution.index(edge2_start)
    
    old_distance = distance_matrix[edge1_start][edge1_end] + distance_matrix[edge2_start][edge2_end]
    new_distance = distance_matrix[edge1_start][edge2_start] + distance_matrix[edge1_end][edge2_end]
    
    delta = old_distance - new_distance


    if idx_edge1_start < idx_edge2_start:
        new_solution = solution[:idx_edge1_start+1] + solution[idx_edge1_start+1:idx_edge2_start+1][::-1] + \
                       solution[idx_edge2_start+1:] 
    else:
        new_solution = solution[:idx_edge2_start+1] + solution[idx_edge2_start+1:idx_edge1_start+1][::-1] + \
                       solution[idx_edge1_start+1:]
    return delta, new_solution

In [ ]:
# @memory.cache
def greedy_local_search(solution, distance_matrix, costs, flag):
    while True:
        neighborhood = get_node_moves(solution, flag) + get_edge_moves(solution)
        np.random.shuffle(neighborhood)
        
        for i, neighbor in enumerate(neighborhood):
            if isinstance(neighbor[0], np.integer) or isinstance(neighbor[0], int):
                delta, new_solution = compute_node_swap_delta(solution, neighbor, distance_matrix, costs, flag)
            else:
                delta, new_solution = compute_edge_swap_delta(solution, neighbor, distance_matrix)
            if delta > 0:
                # print(neighbor, delta)
                solution = new_solution[:]
                break
            # print(i, len(neighborhood))
        else:
            # total_cost = sum(distance_matrix[solution[i]][solution[i+1]]+costs[i+1] for i in range(len(solution) - 1))
            # total_cost += distance_matrix[solution[-1]][solution[0]] + costs[0]
            return solution#, total_cost

In [ ]:
# @memory.cache
def steepest_local_search(solution, distance_matrix, costs, flag):
    while True:
        neighborhood = get_node_moves(solution, flag) + get_edge_moves(solution)
        
        best_delta, best_solution = 0, None
        for i, neighbor in enumerate(neighborhood):
            if isinstance(neighbor[0], np.integer) or isinstance(neighbor[0], int):
                delta, new_solution = compute_node_swap_delta(solution, neighbor, distance_matrix, costs, flag)
            else:
                delta, new_solution = compute_edge_swap_delta(solution, neighbor, distance_matrix)
            if delta > best_delta:
                best_delta = delta
                best_solution = new_solution[:]
            # print(i, len(neighborhood))
        
        if best_solution is not None:
            solution = best_solution[:]
            continue
        
        # total_cost = sum(distance_matrix[solution[i]][solution[i+1]]+costs[i+1] for i in range(len(solution) - 1))
        # total_cost += distance_matrix[solution[-1]][solution[0]] + costs[0]
        return solution#, total_cost

In [ ]:
def greedy_2_regret_weighted(distance_matrix, current_node_index=None, regret_weight=0.5):
    num_nodes = len(distance_matrix)
    to_visit = set(range(num_nodes))
    
    if current_node_index is None:
        current_node_index = np.random.choice(list(to_visit))
    
    current_node = current_node_index
    solution = [current_node]
    to_visit.remove(current_node_index)
    
    while len(solution) != len(distance_matrix) // 2:
        max_weighted_sum = float('-inf')
        best_node = None
        best_insertion_point = None

        for node in to_visit:
            insertion_costs = []
            for i in range(len(solution) - 1):
                cost = distance_matrix[solution[i]][node] + distance_matrix[node][solution[i+1]] - distance_matrix[solution[i]][solution[i+1]]
                insertion_costs.append((cost, i))

            insertion_costs.append((distance_matrix[solution[-1]][node] + distance_matrix[node][solution[0]] - distance_matrix[solution[-1]][solution[0]], len(solution) - 1))

            insertion_costs.sort(key=lambda x: x[0])

            weighted_sum = 0
            if len(insertion_costs) > 1:
                regret = insertion_costs[1][0] - insertion_costs[0][0]
                objectivee = -insertion_costs[0][0]
                weighted_sum = regret_weight * regret   +    (1 - regret_weight) * objectivee

            if weighted_sum > max_weighted_sum:
                max_weighted_sum = weighted_sum
                best_node = node
                best_insertion_point = insertion_costs[0][1]

        solution.insert(best_insertion_point + 1, best_node)
        to_visit.remove(best_node)

    total_cost = sum(distance_matrix[solution[i]][solution[i+1]] for i in range(len(solution) - 1))
    total_cost += distance_matrix[solution[-1]][solution[0]]

    return solution, total_cost

In [ ]:
columns = ["Algorithm", "TSPA", "TSPB", "TSPC", "TSPD"]
cost_df = pd.DataFrame(columns=columns)
time_df = pd.DataFrame(columns=columns)
best_solutions = {}

search_types = {'Greedy': greedy_local_search, 'Steepest': steepest_local_search}
starting_solutions = {'Greedy2Regret': greedy_2_regret_weighted, 'Random': random_search}
instances = ['TSPA', 'TSPB', 'TSPC', 'TSPD']
path = "../data/"
for search_type in search_types:
    for flag in ('intra', 'inter'):
        for starting_solution in starting_solutions:
            algo_name = f'{search_type}-{flag}-{starting_solution}'
            if algo_name != 'Steepest-inter-Random':
                continue
            new_row = pd.DataFrame({columns[0]: algo_name}, index=[0])
            cost_df = pd.concat([cost_df, new_row], ignore_index=True)
            time_df = pd.concat([time_df, new_row], ignore_index=True)
            best_solutions_tmp = {}
            for instance in instances:
                if instance != 'TSPC':
                    continue
                file_name = f'{path}{instance}.csv'
                df = pd.read_csv(file_name, names=['x', 'y', 'cost'], sep=';')
                distance_matrix = get_distance_matrix(df)
                distance_matrix_with_costs = get_distance_matrix(df, True)
                costs = df.cost.values
                total_costs, solutions, times = list(), list(), list()
                print(algo_name, instance)
                for i in range(200):
                    print(i)
                    solution, _ = starting_solutions[starting_solution](distance_matrix_with_costs)
                    start_time = perf_counter()
                    solution = search_types[search_type](solution, distance_matrix, costs, flag)
                    total_cost = sum(distance_matrix_with_costs[solution[i]][solution[i+1]] for i in range(len(solution) - 1))
                    total_cost += distance_matrix_with_costs[solution[-1]][solution[0]]
                    end_time = perf_counter()
                    total_costs.append(total_cost)
                    solutions.append(solution)
                    times.append(round(end_time - start_time, 5))
                best_solution_idx = np.argmin(total_costs)
                best_solutions_tmp[file_name] = solutions[best_solution_idx]
                cost_df.at[cost_df.index[-1], instance] = f'{np.mean(total_costs)} ({np.min(total_costs)} - {np.max(total_costs)})'
                time_df.at[time_df.index[-1], instance] = f'{np.mean(times)} ({np.min(times)} - {np.max(times)})'
                display(cost_df)
                display(time_df)
            best_solutions[algo_name] = best_solutions_tmp

In [ ]:
# cost_df.to_csv('costs.csv')

In [ ]:
# time_df.to_csv('times.csv')

In [ ]:
display(cost_df)

In [ ]:
display(time_df)

In [ ]:
xxx = {'Greedy-intra-GreedyHeuristic': {
                        '../data/TSPA.csv': [183, 61, 163, 74, 113, 195, 22, 53, 62, 108, 81, 154, 102, 144, 87, 141, 6, 172, 156, 98, 66, 24, 45, 167, 101, 99, 135, 51, 5, 112, 169, 95, 31, 73, 72, 190, 94, 89, 111, 14, 80, 124, 8, 26, 106, 48, 11, 152, 130, 119, 109, 189, 75, 1, 177, 41, 199, 150, 192, 175, 153, 88, 127, 186, 170, 21, 79, 194, 171, 117, 55, 36, 132, 128, 145, 76, 161, 91, 121, 114, 4, 77, 43, 50, 149, 0, 19, 178, 164, 159, 143, 59, 147, 116, 27, 96, 185, 64, 20, 71],
                        '../data/TSPB.csv': [192, 21, 142, 130, 174, 51, 91, 70, 140, 148, 141, 53, 82, 115, 63, 8, 14, 16, 172, 95, 198, 135, 66, 169, 0, 57, 99, 92, 122, 143, 179, 197, 183, 34, 31, 101, 42, 38, 103, 131, 121, 127, 24, 50, 152, 94, 112, 154, 134, 25, 36, 123, 165, 37, 102, 137, 88, 55, 4, 153, 80, 157, 145, 79, 190, 19, 29, 33, 136, 61, 73, 185, 132, 18, 52, 12, 107, 139, 193, 119, 59, 71, 44, 166, 85, 64, 147, 159, 89, 129, 58, 171, 72, 114, 67, 158, 162, 150, 117, 196],
                        '../data/TSPC.csv': [82, 185, 64, 20, 71, 61, 25, 181, 113, 163, 74, 138, 195, 53, 62, 32, 180, 81, 108, 15, 117, 22, 55, 36, 132, 128, 145, 76, 161, 153, 88, 127, 186, 45, 24, 170, 129, 157, 21, 194, 79, 154, 102, 144, 87, 141, 6, 156, 172, 66, 98, 190, 72, 12, 94, 42, 89, 73, 31, 111, 14, 107, 80, 124, 123, 8, 110, 169, 95, 112, 5, 51, 135, 99, 101, 167, 175, 4, 77, 43, 114, 91, 121, 50, 149, 0, 69, 35, 19, 178, 164, 34, 159, 143, 59, 147, 116, 27, 96, 46],
                        '../data/TSPD.csv': [3, 78, 45, 67, 126, 114, 158, 162, 150, 117, 196, 44, 71, 59, 119, 193, 139, 97, 107, 12, 52, 65, 16, 18, 132, 185, 73, 61, 136, 33, 29, 168, 19, 190, 198, 135, 86, 79, 145, 157, 80, 153, 4, 55, 88, 36, 25, 134, 154, 123, 165, 37, 137, 146, 99, 57, 0, 169, 66, 128, 26, 92, 122, 143, 127, 24, 121, 179, 197, 183, 34, 11, 5, 167, 133, 182, 2, 48, 115, 40, 8, 63, 82, 69, 113, 32, 96, 53, 142, 130, 141, 148, 140, 188, 161, 174, 51, 70, 91, 156]
    
},
       'Greedy-intra-Random': {
                        '../data/TSPA.csv': [183, 163, 74, 155, 62, 32, 120, 15, 171, 104, 184, 87, 144, 67, 24, 186, 45, 167, 83, 75, 189, 126, 100, 134, 135, 51, 5, 112, 68, 66, 190, 72, 12, 89, 179, 73, 95, 169, 110, 124, 123, 8, 13, 52, 33, 92, 48, 106, 198, 11, 152, 188, 41, 137, 199, 192, 187, 175, 153, 114, 2, 4, 77, 43, 91, 50, 149, 35, 69, 115, 38, 49, 151, 76, 136, 54, 36, 55, 22, 18, 113, 181, 25, 10, 84, 118, 128, 40, 65, 3, 164, 159, 143, 59, 63, 96, 46, 64, 20, 71, 183],
                        '../data/TSPB.csv': [175, 46, 62, 57, 15, 0, 111, 164, 169, 144, 128, 26, 197, 143, 179, 1, 31, 101, 42, 103, 131, 121, 127, 50, 112, 49, 43, 99, 137, 41, 37, 134, 68, 88, 4, 153, 80, 145, 6, 33, 73, 61, 90, 185, 81, 18, 132, 170, 189, 87, 181, 176, 159, 64, 89, 58, 114, 162, 150, 158, 67, 78, 21, 142, 174, 51, 91, 70, 188, 140, 148, 141, 13, 53, 149, 69, 30, 40, 63, 82, 60, 191, 44, 71, 104, 97, 107, 12, 52, 173, 14, 16, 172, 54, 75, 105, 118, 95, 22, 19, 19],
                        '../data/TSPC.csv': [111, 107, 105, 8, 52, 148, 48, 92, 11, 152, 16, 130, 177, 39, 150, 77, 4, 114, 91, 50, 149, 69, 35, 19, 115, 0, 145, 65, 164, 178, 34, 158, 159, 143, 59, 96, 46, 185, 64, 140, 74, 113, 181, 25, 57, 23, 10, 128, 132, 30, 195, 22, 117, 28, 171, 108, 18, 93, 183, 103, 7, 32, 180, 131, 81, 154, 133, 44, 102, 144, 87, 79, 157, 170, 161, 153, 186, 24, 141, 172, 98, 190, 122, 12, 165, 5, 51, 99, 101, 175, 83, 126, 189, 109, 119, 134, 196, 169, 110, 191, 111],
                        '../data/TSPD.csv': [58, 129, 64, 159, 176, 181, 170, 76, 185, 132, 189, 187, 87, 28, 166, 71, 44, 117, 191, 196, 139, 97, 12, 52, 173, 65, 16, 18, 33, 199, 79, 151, 157, 160, 153, 55, 137, 102, 37, 165, 123, 68, 194, 23, 125, 93, 131, 103, 38, 121, 127, 179, 197, 183, 34, 11, 5, 169, 164, 57, 15, 98, 135, 198, 46, 19, 95, 172, 54, 105, 2, 163, 40, 20, 82, 115, 30, 69, 184, 32, 149, 53, 60, 192, 13, 141, 148, 140, 161, 174, 83, 70, 51, 39, 155, 45, 67, 158, 114, 72, 72]
       },
       'Greedy-inter-GreedyHeuristic': {
                            '../data/TSPA.csv': [94, 72, 190, 98, 66, 156, 6, 24, 141, 144, 87, 79, 194, 21, 171, 154, 81, 62, 108, 15, 117, 53, 22, 195, 55, 36, 132, 128, 25, 181, 113, 74, 163, 61, 71, 20, 64, 185, 96, 27, 116, 147, 59, 143, 159, 164, 178, 19, 0, 149, 50, 121, 91, 114, 4, 77, 43, 192, 175, 153, 88, 127, 186, 45, 167, 101, 60, 174, 199, 41, 177, 1, 75, 189, 109, 130, 152, 11, 48, 106, 26, 119, 134, 99, 135, 51, 5, 112, 95, 169, 110, 8, 123, 124, 80, 14, 111, 31, 73, 12],
                            '../data/TSPB.csv': [158, 162, 150, 44, 117, 196, 192, 21, 142, 130, 174, 51, 91, 70, 140, 148, 141, 53, 69, 115, 82, 63, 8, 14, 16, 172, 95, 163, 182, 2, 5, 34, 183, 197, 31, 101, 42, 38, 103, 131, 24, 127, 121, 179, 143, 122, 92, 26, 66, 169, 99, 50, 112, 154, 25, 36, 165, 37, 137, 88, 55, 4, 153, 145, 157, 80, 57, 0, 135, 198, 190, 19, 29, 33, 136, 61, 73, 185, 132, 18, 52, 12, 107, 139, 193, 119, 59, 71, 166, 85, 64, 147, 159, 89, 129, 58, 171, 72, 114, 67],
                            '../data/TSPC.csv': [131, 180, 32, 93, 62, 108, 171, 15, 117, 53, 22, 55, 195, 74, 163, 113, 36, 132, 128, 164, 178, 19, 35, 69, 0, 149, 50, 121, 91, 114, 175, 2, 4, 77, 43, 192, 150, 199, 39, 174, 137, 41, 177, 1, 75, 189, 109, 119, 130, 152, 11, 48, 92, 26, 8, 110, 169, 95, 80, 31, 73, 89, 42, 94, 12, 72, 98, 156, 172, 6, 66, 190, 112, 5, 51, 135, 99, 101, 9, 60, 167, 153, 88, 127, 45, 186, 170, 129, 157, 21, 194, 79, 87, 141, 144, 102, 44, 133, 154, 81],
                            '../data/TSPD.csv': [87, 147, 159, 64, 129, 89, 58, 171, 72, 114, 85, 166, 28, 59, 119, 193, 71, 44, 162, 150, 117, 196, 192, 21, 138, 142, 130, 161, 174, 188, 140, 148, 141, 53, 96, 32, 113, 69, 115, 82, 63, 8, 14, 84, 139, 97, 107, 12, 52, 132, 18, 16, 172, 95, 19, 190, 198, 135, 128, 66, 169, 0, 57, 99, 122, 143, 179, 121, 127, 24, 50, 94, 112, 154, 134, 25, 36, 194, 123, 165, 37, 146, 137, 88, 55, 4, 153, 80, 157, 145, 79, 136, 61, 73, 185, 47, 189, 170, 181, 187]
       },
       
    'Greedy-inter-Random': { '../data/TSPA.csv': [159, 143, 164, 178, 19, 0, 149, 50, 43, 77, 4, 114, 121, 91, 161, 76, 128, 132, 36, 55, 22, 18, 53, 117, 15, 108, 171, 194, 79, 21, 170, 153, 88, 127, 186, 45, 101, 167, 175, 192, 199, 41, 1, 177, 137, 174, 75, 189, 109, 119, 130, 152, 11, 106, 48, 92, 26, 8, 124, 80, 169, 95, 135, 51, 112, 73, 31, 94, 72, 190, 98, 66, 156, 6, 24, 141, 87, 144, 154, 81, 180, 32, 62, 93, 155, 195, 74, 163, 113, 181, 61, 71, 20, 64, 185, 116, 27, 147, 96, 59],
                                '../data/TSPB.csv':[185, 47, 189, 170, 181, 147, 159, 64, 129, 89, 58, 72, 114, 162, 150, 67, 156, 91, 51, 174, 140, 148, 141, 130, 142, 53, 21, 192, 196, 117, 44, 71, 59, 119, 193, 139, 107, 12, 52, 132, 18, 29, 19, 190, 198, 135, 95, 172, 16, 8, 63, 82, 115, 2, 182, 133, 167, 5, 128, 34, 183, 197, 92, 122, 143, 179, 31, 101, 38, 103, 131, 121, 127, 24, 50, 99, 66, 169, 0, 57, 137, 37, 165, 123, 154, 134, 25, 36, 88, 55, 4, 153, 80, 157, 145, 79, 136, 61, 73, 33],
                                '../data/TSPC.csv': [31, 73, 95, 169, 139, 134, 99, 135, 196, 51, 5, 112, 72, 98, 66, 172, 156, 6, 141, 87, 144, 102, 154, 81, 108, 62, 53, 195, 138, 113, 74, 163, 61, 71, 20, 64, 185, 116, 27, 147, 96, 59, 143, 159, 178, 164, 128, 132, 36, 55, 22, 18, 117, 15, 171, 104, 194, 79, 21, 157, 170, 186, 127, 88, 153, 161, 76, 0, 19, 69, 149, 50, 121, 91, 114, 4, 77, 192, 199, 137, 41, 177, 1, 75, 189, 109, 119, 130, 152, 11, 160, 106, 48, 92, 26, 8, 124, 80, 111, 14],
                                '../data/TSPD.csv': [193, 119, 59, 71, 44, 117, 196, 60, 21, 192, 150, 162, 67, 156, 91, 70, 51, 174, 188, 140, 148, 141, 130, 142, 53, 82, 63, 115, 40, 16, 172, 95, 19, 190, 198, 135, 169, 66, 128, 5, 34, 183, 197, 92, 122, 143, 179, 31, 101, 38, 103, 131, 121, 127, 24, 50, 43, 99, 77, 0, 57, 137, 37, 165, 154, 134, 25, 36, 88, 153, 80, 157, 145, 79, 136, 61, 73, 185, 132, 29, 18, 65, 52, 14, 8, 84, 139, 97, 107, 12, 189, 147, 159, 64, 129, 89, 58, 72, 114, 166]
     
},
       'Steepest-intra-GreedyHeuristic TSPA': {
                                '../data/TSPA.csv': [172, 66, 98, 190, 72, 94, 89, 73, 31, 111, 14, 80, 124, 8, 110, 169, 95, 112, 5, 51, 135, 99, 134, 119, 26, 106, 48, 11, 152, 130, 109, 189, 75, 1, 177, 41, 199, 150, 192, 60, 101, 167, 45, 186, 127, 88, 153, 175, 114, 4, 77, 43, 121, 91, 50, 149, 0, 19, 178, 164, 159, 143, 59, 147, 116, 27, 96, 185, 64, 20, 71, 61, 163, 74, 113, 181, 37, 128, 132, 36, 55, 195, 22, 53, 117, 15, 108, 62, 81, 154, 171, 21, 194, 79, 87, 144, 141, 24, 6, 156],
                                '../data/TSPB.csv': [196, 192, 21, 142, 130, 174, 51, 91, 70, 140, 148, 141, 53, 82, 115, 63, 8, 14, 16, 172, 95, 198, 135, 66, 169, 0, 57, 99, 92, 122, 143, 179, 197, 183, 34, 31, 101, 42, 38, 103, 131, 121, 127, 24, 50, 152, 94, 112, 154, 134, 25, 36, 123, 165, 37, 102, 137, 88, 55, 4, 153, 80, 157, 145, 79, 190, 19, 29, 33, 136, 61, 73, 185, 132, 18, 52, 12, 107, 139, 193, 119, 59, 71, 166, 85, 64, 147, 159, 89, 129, 58, 171, 72, 114, 67, 158, 162, 150, 44, 117],
                                '../data/TSPC.csv': [25, 181, 113, 163, 74, 138, 195, 53, 62, 108, 15, 117, 22, 55, 36, 132, 145, 76, 161, 153, 88, 127, 186, 45, 24, 170, 129, 157, 21, 194, 79, 154, 102, 144, 87, 141, 6, 156, 172, 66, 98, 190, 72, 12, 94, 42, 89, 73, 31, 111, 14, 107, 80, 124, 123, 8, 110, 169, 95, 112, 5, 51, 135, 99, 101, 167, 175, 114, 4, 77, 43, 121, 91, 50, 149, 0, 69, 35, 19, 178, 164, 34, 40, 128, 118, 84, 37, 159, 143, 59, 147, 116, 27, 96, 46, 185, 64, 20, 71, 61],
                                '../data/TSPD.csv': [85, 110, 166, 71, 59, 119, 193, 139, 97, 107, 12, 52, 132, 185, 73, 18, 29, 33, 19, 190, 198, 135, 86, 79, 145, 157, 80, 153, 4, 55, 88, 165, 37, 137, 146, 99, 57, 0, 169, 66, 128, 26, 92, 122, 143, 127, 24, 121, 179, 197, 183, 34, 11, 5, 167, 133, 2, 182, 163, 95, 172, 16, 14, 8, 63, 82, 115, 69, 113, 32, 96, 53, 141, 148, 140, 188, 174, 161, 130, 142, 138, 21, 192, 196, 117, 44, 150, 162, 158, 126, 45, 67, 114, 72, 58, 129, 89, 159, 147, 64]
       },
       'Steepest-intra-Random': {
                                '../data/TSPA.csv': [77, 43, 121, 50, 149, 0, 193, 58, 178, 40, 118, 36, 145, 54, 55, 22, 53, 47, 155, 163, 166, 25, 159, 143, 147, 116, 96, 46, 82, 185, 71, 103, 182, 7, 146, 93, 62, 120, 108, 171, 104, 133, 144, 141, 194, 157, 170, 127, 167, 17, 119, 100, 33, 110, 169, 95, 196, 51, 5, 68, 66, 6, 172, 156, 72, 94, 12, 179, 89, 42, 111, 14, 107, 31, 124, 105, 8, 26, 52, 148, 106, 198, 48, 92, 130, 16, 162, 11, 152, 1, 177, 41, 137, 199, 39, 174, 83, 175, 2, 4],
                                '../data/TSPB.csv': [64, 129, 89, 159, 147, 187, 87, 100, 189, 59, 97, 107, 139, 60, 191, 196, 162, 71, 110, 58, 72, 114, 39, 91, 51, 156, 3, 155, 9, 192, 21, 138, 142, 148, 141, 13, 53, 96, 113, 115, 82, 63, 163, 2, 182, 54, 133, 167, 178, 128, 66, 169, 164, 92, 143, 197, 183, 31, 101, 42, 103, 121, 179, 24, 108, 50, 49, 41, 37, 23, 134, 25, 36, 88, 55, 160, 180, 137, 62, 77, 57, 46, 198, 86, 151, 79, 136, 73, 76, 185, 81, 33, 6, 29, 18, 14, 8, 84, 52, 132],
                                '../data/TSPC.csv': [96, 70, 185, 20, 71, 183, 182, 146, 180, 131, 81, 133, 102, 144, 141, 6, 156, 172, 66, 98, 190, 122, 94, 179, 73, 111, 14, 107, 191, 169, 110, 26, 148, 92, 48, 160, 11, 152, 177, 75, 189, 109, 134, 196, 51, 135, 99, 101, 60, 17, 83, 97, 173, 174, 137, 41, 199, 150, 192, 187, 4, 50, 121, 114, 2, 175, 167, 170, 194, 117, 15, 120, 53, 22, 47, 113, 142, 166, 181, 85, 57, 37, 84, 132, 36, 55, 54, 136, 193, 38, 0, 86, 35, 58, 164, 3, 40, 34, 159, 143],
                                '../data/TSPD.csv': [80, 180, 73, 185, 170, 181, 176, 159, 64, 129, 89, 72, 85, 110, 114, 126, 67, 45, 78, 3, 91, 51, 83, 161, 140, 141, 130, 142, 149, 184, 32, 113, 69, 115, 82, 138, 60, 192, 117, 44, 166, 71, 193, 119, 59, 104, 109, 12, 173, 65, 18, 177, 33, 168, 120, 19, 74, 172, 163, 17, 182, 2, 105, 133, 54, 167, 144, 178, 34, 101, 38, 131, 93, 108, 112, 25, 36, 68, 4, 153, 62, 57, 0, 77, 41, 37, 165, 102, 49, 50, 24, 121, 127, 122, 197, 26, 66, 169, 98, 198]
       },
       'Steepest-inter-GreedyHeuristic': {
                                '../data/TSPA.csv': [172, 66, 98, 190, 72, 94, 12, 73, 31, 111, 14, 80, 124, 8, 110, 169, 95, 112, 5, 51, 135, 99, 134, 119, 26, 106, 48, 11, 152, 130, 109, 189, 75, 1, 177, 41, 199, 150, 192, 60, 101, 167, 45, 186, 127, 88, 153, 175, 114, 4, 77, 43, 121, 91, 50, 149, 0, 19, 178, 164, 159, 143, 59, 147, 116, 27, 96, 185, 64, 20, 71, 61, 163, 74, 113, 181, 25, 128, 132, 36, 55, 195, 22, 53, 117, 15, 108, 62, 81, 154, 171, 21, 194, 79, 87, 144, 141, 24, 6, 156],
                                '../data/TSPB.csv': [193, 139, 107, 12, 52, 132, 185, 73, 61, 136, 33, 29, 18, 16, 172, 95, 19, 190, 198, 135, 0, 57, 80, 157, 145, 153, 4, 55, 88, 137, 37, 165, 36, 25, 134, 154, 50, 99, 169, 66, 26, 92, 122, 143, 127, 24, 121, 131, 103, 38, 42, 101, 31, 179, 197, 183, 34, 5, 2, 182, 163, 40, 8, 63, 82, 115, 69, 53, 141, 148, 140, 70, 91, 51, 174, 130, 142, 21, 192, 196, 117, 44, 150, 162, 158, 67, 114, 72, 171, 58, 129, 89, 159, 147, 64, 85, 166, 71, 59, 119],
                                '../data/TSPC.csv': [81, 171, 108, 62, 15, 117, 53, 22, 55, 195, 74, 163, 113, 132, 128, 40, 164, 178, 19, 35, 69, 0, 149, 50, 121, 91, 114, 175, 2, 4, 77, 43, 192, 150, 199, 39, 174, 137, 41, 177, 1, 75, 189, 109, 130, 152, 11, 160, 106, 48, 92, 26, 119, 134, 139, 95, 169, 110, 8, 80, 31, 73, 89, 42, 94, 12, 72, 98, 156, 172, 6, 66, 190, 112, 5, 51, 135, 99, 101, 9, 60, 167, 153, 88, 127, 45, 186, 170, 129, 157, 21, 194, 79, 87, 141, 144, 102, 44, 133, 154],
                                '../data/TSPD.csv': [126, 158, 162, 150, 44, 117, 196, 192, 21, 138, 142, 130, 161, 174, 188, 140, 148, 141, 53, 96, 32, 113, 69, 115, 82, 63, 8, 14, 16, 172, 95, 163, 182, 2, 133, 167, 5, 11, 34, 183, 197, 179, 121, 24, 127, 143, 122, 92, 26, 128, 66, 169, 0, 57, 99, 146, 137, 37, 165, 123, 154, 134, 25, 36, 88, 55, 4, 153, 80, 157, 145, 79, 86, 135, 198, 190, 19, 33, 29, 18, 73, 185, 132, 52, 12, 107, 139, 97, 59, 119, 193, 71, 64, 159, 89, 129, 72, 114, 67, 45]
       },
    'Steepest-inter-Random': {'../data/TSPA.csv': [21, 171, 15, 117, 53, 22, 195, 55, 36, 132, 128, 37, 159, 143, 59, 147, 96, 185, 64, 20, 71, 61, 181, 113, 74, 163, 62, 32, 180, 108, 81, 154, 102, 144, 87, 141, 6, 156, 172, 66, 98, 190, 72, 94, 42, 89, 12, 73, 31, 111, 14, 80, 124, 123, 8, 169, 95, 112, 51, 135, 134, 119, 26, 106, 160, 11, 152, 48, 92, 130, 109, 189, 75, 1, 177, 41, 199, 192, 4, 114, 77, 43, 50, 149, 19, 178, 164, 0, 121, 91, 153, 175, 60, 101, 167, 88, 127, 186, 194, 79],
                              '../data/TSPB.csv': [2, 182, 163, 172, 16, 65, 52, 14, 8, 63, 115, 82, 53, 142, 130, 141, 148, 140, 174, 51, 70, 91, 156, 192, 196, 117, 44, 150, 162, 158, 67, 114, 72, 58, 89, 159, 147, 64, 129, 166, 71, 59, 119, 193, 139, 107, 12, 189, 181, 170, 185, 132, 18, 33, 73, 136, 79, 145, 157, 80, 153, 4, 55, 88, 137, 37, 165, 36, 25, 154, 112, 50, 24, 127, 121, 131, 103, 38, 101, 31, 179, 143, 122, 92, 197, 183, 34, 5, 128, 66, 169, 0, 99, 57, 135, 198, 190, 19, 95, 133],
                              '../data/TSPC.csv': [22, 55, 195, 53, 18, 15, 108, 180, 32, 62, 93, 155, 138, 74, 163, 113, 61, 71, 20, 64, 185, 96, 27, 116, 147, 59, 143, 159, 178, 164, 40, 128, 118, 132, 36, 76, 0, 19, 69, 149, 50, 121, 91, 153, 88, 127, 186, 167, 101, 175, 114, 4, 77, 43, 192, 150, 199, 39, 137, 41, 1, 177, 174, 75, 189, 109, 119, 152, 11, 160, 106, 48, 92, 26, 8, 105, 110, 169, 95, 196, 135, 51, 112, 73, 31, 89, 94, 72, 190, 98, 66, 6, 141, 144, 87, 79, 194, 21, 171, 117],
                              '../data/TSPD.csv': [ 73,  81, 185, 132, 189, 181, 147, 159,  64, 129,  89,  58,  72,114,  67,  45, 126, 158, 162, 150,  44,  71, 119,  59,  12, 107,
       139, 117, 196,  60, 192,  21, 138, 142, 174,  51,  91,  70, 140,
       148, 130, 141,  53,  69, 115,  82,  63,   8,  14,  16,  65,  52,
        18,  29,  33,  19, 190, 198, 135, 128,   5,  34, 183, 197,  31,
       101,  38, 103, 131, 121,  24, 127, 179, 143, 122,  92,  66, 169,
         0,  57,  77,  99, 146, 137,  37, 165, 123, 154, 134,  25,  36,
        88,  55, 153,  80, 157, 145,  79, 136,  61]
    }                           
}

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [ ]:
best_solutions = xxx

In [ ]:
for algo in best_solutions:
    fig, ax = plt.subplots(2, 2, figsize=(25, 25))
    ax = ax.flatten()
    idx = 0
    fig.suptitle(algo, fontsize=40)
    for instance in best_solutions[algo]:
        solution = best_solutions[algo][instance]
        df = pd.read_csv(instance, names=['x', 'y', 'cost'], sep=';')
        weights = df['cost']
        cmap = plt.cm.viridis
        norm = mcolors.Normalize(vmin=min(weights), vmax=max(weights))
        ax[idx].set_title(instance.replace('../data/', '').replace('.csv', ''), fontsize=30)
        for i in range(len(df)):
            x, y, cost = df.iloc[i]['x'], df.iloc[i]['y'], df.iloc[i]['cost']
            ax[idx].plot(x, y, "o", markersize=20, color=cmap(norm(cost)))
        for i in range(len(solution)-1):
            x, y, cost = df.iloc[solution[i]]['x'], df.iloc[solution[i]]['y'], df.iloc[solution[i]]['cost']
            x_next, y_next = df.iloc[solution[i+1]]['x'], df.iloc[solution[i+1]]['y']
            ax[idx].plot((x, x_next), (y, y_next), "-", color='black')
        x, y = df.iloc[solution[0]]['x'], df.iloc[solution[0]]['y']
        ax[idx].plot((x, x_next), (y, y_next), "-", color='black')
        
        axins = ax[idx].inset_axes([1.05, 0.1, 0.05, 0.6], transform=ax[idx].transAxes)
        gradient = np.linspace(0, 1, 256).reshape(-1, 1)

        axins.imshow(gradient, aspect='auto', cmap=cmap, origin='lower', extent=[0, 1, min(weights), max(weights)])
        axins.xaxis.set_visible(False)
        
        idx += 1
        
    plt.savefig(f'./plots/{algo}.png', dpi=300)
    plt.show()
    